# The VIX Index Calculation:  Step-by-Step 

$\sigma^2=\frac{2}{T}\sum_i\frac{\varDelta K_i}{K_i^2}e^{RT}Q(K_i)-\frac{1}{T}[\frac{F}{K_0}-1]^2$

$\sigma = \frac{VIX}{100}$

$T$: Time to expiration

$F$: Forward index level derived from index option prices

$K_0$: First strike below the forward index level, $F$

$K_i$: Strike price of ith out-of-the-money option; a call if $K_i > K_0$ and a put if $K_i < K_0$; both put and call if $K_i = K_0$.

$\varDelta K_i$: Interval between strike prices - half the difference between the strike on either side of $K_i$

$\varDelta K_i=\frac{K_{i+1}-K_{i-1}}{2}$

$R$: Risk-free interest rate to expiration

$Q(K_i)$: The midpoint of the bid-ask spread for each option with strike $K_i$.

## Getting Started

$T =\frac{M_{Current day} + M_{Settlement day} + M_{Other days}}{Minutes in a year}$

$M_{Current day}$: minutes remaining until midnight of the current day

$M_{Settlement day}$: minutes from midnight until 9:30 a.m. ET for 'standard' SPX expirations; or minutes from midnight until 4:00 p.m. ET for 'weekly' SPX expirations

$M_{Other days}$: total minutes in the days between current day and expiration day

In [89]:
def getTime(M_current, M_settlement, M_other, M_in_year):
    return (M_current + M_settlement + M_other) / M_in_year

In [5]:
import datetime

$M_{currentday}$

In [83]:
today = datetime.datetime.now()
tomorrow = datetime.datetime(today.year, today.month, today.day + 1, 0, 0, 0, 0)
diff = tomorrow - today
M_current = diff.seconds / 60
print('M_current: {} mins'.format(M_current))

M_current: 371.95 mins


$M_{settlement day}$

In [84]:
M_settlement_near_term = datetime.datetime(today.year, today.month + 1, 1, 9, 30, 0, 0) - datetime.datetime(today.year, today.month + 1, 1, 0, 0, 0, 0)
M_settlement_near_term = M_settlement_near_term.total_seconds() / 60

M_settlement_next_term = datetime.datetime(today.year, today.month + 2, 1, 9, 30, 0, 0) - datetime.datetime(today.year, today.month + 2, 1, 0, 0, 0, 0)
M_settlement_next_term = M_settlement_next_term.total_seconds() / 60
print('M_settlement_near_term: {} mins\tM_settlement_next_term: {} mins'.format(M_settlement_near_term, M_settlement_next_term))

M_settlement_near_term: 570.0 mins	M_settlement_next_term: 570.0 mins


$M_{other days}$

In [92]:
M_other_near_term = datetime.datetime(today.year, today.month + 1, 1, 0, 0, 0, 0) - today
M_other_next_term = datetime.datetime(today.year, today.month + 2, 1, 0, 0, 0, 0) - today

M_other_near_term = M_other_near_term.days * 24 * 60
M_other_next_term = M_other_next_term.days * 24 * 60
print('M_other_near_term: {} mins\tM_other_next_term: {} mins'.format(M_other_near_term, M_other_next_term))

M_other_near_term: 8640 mins	M_other_next_term: 51840 mins


$M_{year}$

In [87]:
M_year = 365 * 24 * 60
print('M_year: {} mins'.format(M_year))

M_year: 525600 mins


$T_1$ and $T_2$ - Time weights

In [95]:
T1 = getTime(M_current, M_settlement_near_term, M_other_near_term, M_year)
T2 = getTime(M_current, M_settlement_next_term, M_other_next_term, M_year)
print('T1: {} mins\tT2: {} mins'.format(T1, T2))

T1: 0.018230498477929985 mins	T2: 0.10042227929984779 mins


$R$: Risk-free rate

$R_1$: for the near-term options

$R_2$: for the next-term options

In [97]:
R1 = 0.0165
R2 = 0.0151

## Step 1: Select the options to be used in the VIX Index calculation 

### Load Option data

In [99]:
import pandas as pd

In [100]:
df = pd.read_csv('Option_price.csv')

In [127]:
ontrade = df[(df['delist_reason'] != '合约到期') & (df['underlying_name'] == '50ETF')]
ontrade.head()

,Unnamed: 0,id,code,trading_code,name,contract_type,exchange_code,currency_id,underlying_symbol,underlying_name,...,list_price,high_limit,low_limit,expire_date,last_trade_date,exercise_date,delivery_date,is_adjust,delist_date,delist_reason
1908,1908,18427,10001909.XSHG,510050C2003A02750,50ETF购3月2706A,CO,XSHG,CNY,510050.XSHG,50ETF,...,0.3401,0.6353,0.0449,2020-03-25,2020-03-25,2020-03-25,2020-03-26,1,NaN,NaN
1909,1909,18428,10001910.XSHG,510050C2003A02800,50ETF购3月2755A,CO,XSHG,CNY,510050.XSHG,50ETF,...,0.3085,0.6037,0.0133,2020-03-25,2020-03-25,2020-03-25,2020-03-26,1,NaN,NaN
1910,1910,18429,10001911.XSHG,510050C2003A02850,50ETF购3月2804A,CO,XSHG,CNY,510050.XSHG,50ETF,...,0.2814,0.5766,0.0001,2020-03-25,2020-03-25,2020-03-25,2020-03-26,1,NaN,NaN
1911,1911,18430,10001912.XSHG,510050C2003A02900,50ETF购3月2853A,CO,XSHG,CNY,510050.XSHG,50ETF,...,0.2542,0.5494,0.0001,2020-03-25,2020-03-25,2020-03-25,2020-03-26,1,NaN,NaN
1912,1912,18431,10001913.XSHG,510050C2003A02950,50ETF购3月2903A,CO,XSHG,CNY,510050.XSHG,50ETF,...,0.2285,0.5237,0.0001,2020-03-25,2020-03-25,2020-03-25,2020-03-26,1,NaN,NaN


In [139]:
expire_date_near_term = '2020-03-25'
expire_date_next_term = '2020-04-22'

In [143]:
Near_Term_Options_table = ontrade[ontrade['expire_date'] == expire_date_near_term]
Next_Term_Options_table = ontrade[ontrade['expire_date'] == expire_date_next_term]

In [192]:
# Near Term Call Options
Near_Term_Call_Options = Near_Term_Options_table[(Near_Term_Options_table['contract_type'] == 'CO')]
Near_Term_Call_Options = Near_Term_Call_Options[['name', 'list_price']]
# Next Term Call Options
Next_Term_Call_Options = Next_Term_Options_table[(Next_Term_Options_table['contract_type'] == 'CO')]
Next_Term_Call_Options = Next_Term_Call_Options[['name', 'list_price']]

In [151]:
Near_Term_Call_Options.head()

,name,list_price
1908,50ETF购3月2706A,0.3401
1909,50ETF购3月2755A,0.3085
1910,50ETF购3月2804A,0.2814
1911,50ETF购3月2853A,0.2542
1912,50ETF购3月2903A,0.2285


In [193]:
Next_Term_Call_Options.head()

,name,list_price
2332,50ETF购4月2700,0.2442
2333,50ETF购4月2750,0.2009
2334,50ETF购4月2800,0.1651
2335,50ETF购4月2850,0.1349
2336,50ETF购4月2900,0.1060


In [194]:
# Near Term Put Options
Near_Term_Put_Options = Near_Term_Options_table[(Near_Term_Options_table['contract_type'] == 'PO')]
Near_Term_Put_Options = Near_Term_Put_Options[['name', 'list_price']]
# Next Term Put Options
Next_Term_Put_Options = Next_Term_Options_table[(Next_Term_Options_table['contract_type'] == 'PO')]
Next_Term_Put_Options = Next_Term_Put_Options[['name', 'list_price']]

In [195]:
Near_Term_Put_Options.head()

,name,list_price
1917,50ETF沽3月2706A,0.0897
1918,50ETF沽3月2755A,0.1089
1919,50ETF沽3月2804A,0.1297
1920,50ETF沽3月2853A,0.1516
1921,50ETF沽3月2903A,0.1761


In [196]:
Next_Term_Put_Options.head()

,name,list_price
2341,50ETF沽4月2700,0.0357
2342,50ETF沽4月2750,0.0463
2343,50ETF沽4月2800,0.0613
2344,50ETF沽4月2850,0.0778
2345,50ETF沽4月2900,0.0981


In [197]:
import re
def getStrikePrices(names):
    # Create pattern for filter out strike prices
    pattern = re.compile('[0-9][0-9][0-9][0-9]')
    strike_prices = []
    for name in names:
        strike_price = re.findall(pattern, name)[0]
        strike_prices.append(float(strike_price))
    return strike_prices

In [200]:
# Add strike prices back to call options table
Near_Term_Call_Options['Strike Price'] = getStrikePrices(Near_Term_Call_Options['name'].values)
Next_Term_Call_Options['Strike Price'] = getStrikePrices(Next_Term_Call_Options['name'].values)

# Add strike prices back to put options table
Near_Term_Put_Options['Strike Price'] = getStrikePrices(Near_Term_Put_Options['name'].values)
Next_Term_Put_Options['Strike Price'] = getStrikePrices(Next_Term_Put_Options['name'].values)

In [206]:
# Combine Near-term options
NrTCO = Near_Term_Call_Options.set_index('Strike Price').drop(columns='name').rename(columns={'list_price':'Call'})
NrTPO = Near_Term_Put_Options.set_index('Strike Price').drop(columns='name').rename(columns={'list_price':'Put'})
# Combine Near-term options
NtTCO = Next_Term_Call_Options.set_index('Strike Price').drop(columns='name').rename(columns={'list_price':'Call'})
NtTPO = Next_Term_Put_Options.set_index('Strike Price').drop(columns='name').rename(columns={'list_price':'Put'})

In [207]:
Com_Near_Term_Options = NrTCO.join(NrTPO, on='Strike Price')
Com_Next_Term_Options = NtTCO.join(NtTPO, on='Strike Price')

In [210]:
Com_Near_Term_Options['Difference'] = Com_Near_Term_Options['Call'] - Com_Near_Term_Options['Put']
Com_Next_Term_Options['Difference'] = Com_Next_Term_Options['Call'] - Com_Next_Term_Options['Put']

In [267]:
Com_Near_Term_Options.head()

,Call,Put,Difference
Strike Price,,,
2706.0,0.3401,0.0897,0.2504
2755.0,0.3085,0.1089,0.1996
2804.0,0.2814,0.1297,0.1517
2853.0,0.2542,0.1516,0.1026
2903.0,0.2285,0.1761,0.0524


In [266]:
Com_Next_Term_Options.head()

,Call,Put,Difference
Strike Price,,,
2700.0,0.2442,0.0357,0.2085
2750.0,0.2009,0.0463,0.1546
2800.0,0.1651,0.0613,0.1038
2850.0,0.1349,0.0778,0.0571
2900.0,0.1060,0.0981,0.0079


## Determine $F_1$ and $F_2$

In [256]:
search_F_1 = Com_Near_Term_Options.reset_index()
search_F_2 = Com_Next_Term_Options.reset_index()

In [269]:
search_F_2

,Strike Price,Call,Put,Difference
0,2700.0,0.2442,0.0357,0.2085
1,2750.0,0.2009,0.0463,0.1546
2,2800.0,0.1651,0.0613,0.1038
3,2850.0,0.1349,0.0778,0.0571
4,2900.0,0.1060,0.0981,0.0079
5,2950.0,0.0816,0.1225,-0.0409
6,3000.0,0.0623,0.1529,-0.0906
7,3100.0,0.0350,0.2300,-0.1950
8,3200.0,0.0191,0.3146,-0.2955
9,2600.0,0.2485,0.0265,0.2220


In [270]:
search_F_2['Strike Price'].values

array([2700., 2750., 2800., 2850., 2900., 2950., 3000., 3100., 3200.,
       2600., 2650., 3300., 3400., 2500., 2550., 2400., 2450., 2350.])

In [271]:
search_F_2['Difference'].values

array([ 0.2085,  0.1546,  0.1038,  0.0571,  0.0079, -0.0409, -0.0906,
       -0.195 , -0.2955,  0.222 ,  0.1716, -0.3549, -0.3896,  0.1772,
        0.1258,  0.1968,  0.1424,  0.2067])

In [273]:
# Call Option Price - Put Option Price for F1
C_P_Diff_F1 = search_F_1['Difference'].values[0]
Strike_Price_F1 = search_F_1['Strike Price'].values[0]
min_diff = abs(search_F_1['Difference'].values[0])
for i in range(1, len(search_F_1['Difference'].values)):
    if abs(search_F_1['Difference'].values[i]) < min_diff:
        min_diff = abs(search_F_1['Difference'].values[i])
        C_P_Diff_F1 = search_F_1['Difference'].values[i]
        Strike_Price_F1 = search_F_1['Strike Price'].values[i]

# Call Option Price - Put Option Price for F2
C_P_Diff_F2 = search_F_2['Difference'].values[0]
Strike_Price_F2 = search_F_2['Strike Price'].values[0]
min_diff = abs(search_F_2['Difference'].values[0])
for i in range(1, len(search_F_2['Difference'].values)):
    if abs(search_F_2['Difference'].values[i]) < min_diff:
        min_diff = abs(search_F_2['Difference'].values[i])
        C_P_Diff_F2 = search_F_2['Difference'].values[i]
        Strike_Price_F2 = search_F_2['Strike Price'].values[i]

print('Call Option Price - Put Price for F1: {}\tCall Option Price - Put Option Price for F2: {}'.format(C_P_Diff_F1, C_P_Diff_F2))
print('Strike Price for F1: {}\tStrike Price for F2: {}'.format(Strike_Price_F1, Strike_Price_F2))

Call Option Price - Put Price for F1: 0.0039000000000000146	Call Option Price - Put Option Price for F2: 0.00789999999999999
Strike Price for F1: 2952.0	Strike Price for F2: 2900.0


In [274]:
import math

In [275]:
F1 = Strike_Price_F1 + math.exp(R1 * T1) * C_P_Diff_F1
F2 = Strike_Price_F2 + math.exp(R2 * T2) * C_P_Diff_F2
print('F1: {}\tF2: {}'.format(F1, F2))

F1: 2952.0039011733093	F2: 2900.007911988461


## Determine $K_{0,1}$ and $K_{0,2}$

In [288]:
K01 = Strike_Price_F1
K02 = Strike_Price_F2

Select out-of-the-money put options with strike prices < $K0$.  Start with the put strike immediately lower than $K0$ and move to successively lower strike prices.  Exclude any put option that has a bid price equal to zero (i.e., no bid).  As shown below, once two puts with consecutive strike prices are found to have zero bid prices, no puts with lower strikes are considered for inclusion.  (Note that the 1350 and 1355 put options are not included despite having non-zero bid prices.)

## Determine $K_i$: Strike price of ith out-of-the-money option; a call if $K_i > K_0$ and a put if $K_i < K_0$; both put and call if $K_i = K_0$.

In [310]:
Sorted_Com_Near_Term_Options = Com_Near_Term_Options.sort_index()
Sorted_Com_Next_Term_Options = Com_Next_Term_Options.sort_index()

In [311]:
#Determin Ki for Near Term
Near_term_Strike = Sorted_Com_Near_Term_Options.index
Near_term_Option_Type = []
Mid_Point_Price = []
for i in range(0, len(Near_term_Strike)):
    if Near_term_Strike[i] > K01:
        Near_term_Option_Type.append('CO')
        Mid_Point_Price.append(Sorted_Com_Near_Term_Options['Call'].values[i])
    elif Near_term_Strike[i] < K01:
        Near_term_Option_Type.append('PO')
        Mid_Point_Price.append(Sorted_Com_Near_Term_Options['Put'].values[i])
    elif Near_term_Strike[i] == K01:
        Near_term_Option_Type.append('CO&PO')
        Mid_Point_Price.append((Sorted_Com_Near_Term_Options['Put'].values[i + 1] + Sorted_Com_Near_Term_Options['Put'].values[i - 1])/2)
                               
Sorted_Com_Near_Term_Options['Option Type'] = Near_term_Option_Type
Sorted_Com_Near_Term_Options['Midpoint Price'] = Mid_Point_Price

#Determin Ki for Next Term
Next_term_Strike = Sorted_Com_Next_Term_Options.index
Next_term_Option_Type = []
Mid_Point_Price = []
for i in range(0, len(Next_term_Strike)):
    if Next_term_Strike[i] > K01:
        Next_term_Option_Type.append('CO')
        Mid_Point_Price.append(Sorted_Com_Next_Term_Options['Call'].values[i])
    elif Next_term_Strike[i] < K01:
        Next_term_Option_Type.append('PO')
        Mid_Point_Price.append(Sorted_Com_Next_Term_Options['Put'].values[i])
    elif Next_term_Strike[i] == K01:
        Next_term_Option_Type.append('CO&PO')
        Mid_Point_Price.append((Sorted_Com_Next_Term_Options['Put'].values[i + 1] + Sorted_Com_Next_Term_Options['Put'].values[i - 1])/2)
        
Sorted_Com_Next_Term_Options['Option Type'] = Next_term_Option_Type
Sorted_Com_Next_Term_Options['Midpoint Price'] = Mid_Point_Price

### The Midpoint Price is $Q(K)$

In [313]:
Sorted_Com_Near_Term_Options.head()

,Call,Put,Difference,Option Type,Midpoint Price
Strike Price,,,,,
2350.0,0.2190,0.0036,0.2154,PO,0.0036
2400.0,0.2194,0.0067,0.2127,PO,0.0067
2450.0,0.1694,0.0130,0.1564,PO,0.0130
2500.0,0.2180,0.0619,0.1561,PO,0.0619
2550.0,0.1723,0.0781,0.0942,PO,0.0781


In [314]:
Sorted_Com_Next_Term_Options.head()

,Call,Put,Difference,Option Type,Midpoint Price
Strike Price,,,,,
2350.0,0.2472,0.0405,0.2067,PO,0.0405
2400.0,0.2431,0.0463,0.1968,PO,0.0463
2450.0,0.2031,0.0607,0.1424,PO,0.0607
2500.0,0.2452,0.0680,0.1772,PO,0.0680
2550.0,0.2111,0.0853,0.1258,PO,0.0853


## Determine $\varDelta K_i$: Interval between strike prices - half the difference between the strike on either side of $K_i$

In [328]:
# Delta K for Near Term
Near_Term_Delta_K_source = Sorted_Com_Near_Term_Options.index
Near_Term_Delta_K_target = []
Near_Term_Delta_K_target.append((Near_Term_Delta_K_source[1] - Near_Term_Delta_K_source[0]) / 2)

n = Near_Term_Delta_K_source.size
for i in range(1, n - 1):
    delta_k = (Near_Term_Delta_K_source[i + 1] - Near_Term_Delta_K_source[i - 1]) / 2
    Near_Term_Delta_K_target.append(delta_k)
    
Near_Term_Delta_K_target.append((Near_Term_Delta_K_source[n - 1] - Near_Term_Delta_K_source[n - 2]) / 2)
Sorted_Com_Near_Term_Options['Delta K'] = Near_Term_Delta_K_target
    
# Delta K for Near Term
Next_Term_Delta_K_source = Sorted_Com_Next_Term_Options.index
Next_Term_Delta_K_target = []
Next_Term_Delta_K_target.append((Next_Term_Delta_K_source[1] - Next_Term_Delta_K_source[0]) / 2)

n = Next_Term_Delta_K_source.size
for i in range(1, n - 1):
    delta_k = (Next_Term_Delta_K_source[i + 1] - Next_Term_Delta_K_source[i - 1]) / 2
    Next_Term_Delta_K_target.append(delta_k)
    
Next_Term_Delta_K_target.append((Next_Term_Delta_K_source[n - 1] - Next_Term_Delta_K_source[n - 2]) / 2)
Sorted_Com_Next_Term_Options['Delta K'] = Next_Term_Delta_K_target

## Step 2: Calculate volatility for both near-term and next-term options 

In [333]:
Sorted_Com_Near_Term_Options.rename(columns={'Midpoint Price': 'Q_K'}, inplace=True)

In [334]:
Sorted_Com_Next_Term_Options.rename(columns={'Midpoint Price': 'Q_K'}, inplace=True)

In [336]:
Final_Near_Term_Options = Sorted_Com_Near_Term_Options.copy()
Final_Next_Term_Options = Sorted_Com_Next_Term_Options.copy()

$\sigma^2=\frac{2}{T}\sum_i\frac{\varDelta K_i}{K_i^2}e^{RT}Q(K_i)-\frac{1}{T}[\frac{F}{K_0}-1]^2$

$\sigma^2_1$

In [338]:
Final_Near_Term_Options.head()

,Call,Put,Difference,Option Type,Q_K,Delta K
Strike Price,,,,,,
2350.0,0.2190,0.0036,0.2154,PO,0.0036,25.0
2400.0,0.2194,0.0067,0.2127,PO,0.0067,50.0
2450.0,0.1694,0.0130,0.1564,PO,0.0130,50.0
2500.0,0.2180,0.0619,0.1561,PO,0.0619,50.0
2550.0,0.1723,0.0781,0.0942,PO,0.0781,29.0


In [340]:
print('T1={}\tF={}\tK01={}\tR1={}'.format(T1, F1, K01, R1))

T1=0.018230498477929985	F=2952.0039011733093	K01=2952.0	R1=0.0165


In [341]:
Delta_K = Final_Near_Term_Options['Delta K']
K = Final_Near_Term_Options.index
Q_K = Final_Near_Term_Options['Q_K']

In [356]:
sigma_square_1 = (2 / T1 * sum(Delta_K / K**2 * math.exp(R1 * T1) * Q_K)) - 1 / T1 * (F1 / K01 - 1)**2

$\sigma^2_2$

In [357]:
Final_Next_Term_Options.head()

,Call,Put,Difference,Option Type,Q_K,Delta K
Strike Price,,,,,,
2350.0,0.2472,0.0405,0.2067,PO,0.0405,25.0
2400.0,0.2431,0.0463,0.1968,PO,0.0463,50.0
2450.0,0.2031,0.0607,0.1424,PO,0.0607,50.0
2500.0,0.2452,0.0680,0.1772,PO,0.0680,50.0
2550.0,0.2111,0.0853,0.1258,PO,0.0853,50.0


In [358]:
print('T2={}\tF={}\tK02={}\tR2={}'.format(T2, F2, K02, R2))

T2=0.10042227929984779	F=2900.007911988461	K02=2900.0	R2=0.0151


In [359]:
sigma_square_2 = (2 / T2 * sum(Delta_K / K**2 * math.exp(R2 * T2) * Q_K)) - 1 / T2 * (F2 / K02 - 1)**2

In [361]:
print('Sigma_Square_1={}\tSigma_Square_2={}'.format(sigma_square_1, sigma_square_2))

Sigma_Square_1=0.0010154720804418948	Sigma_Square_2=0.0001845713292744068


## Step 3

Calculate the 30-day weighted average of σ21 and σ22.  Then take the square root of that value and multiply by 100 to get the VIX Index value.

$VIX = 100\times\sqrt{\left{T_1\sigma^2_1\left[\frac{N_T_2-N_30}{N_T_2-N_T_1}\right] + T_2\sigma^2_2\left[\frac{N_30-N_T_1}{N_T_2-N_T_1}\right]\right}\times\frac{N_365}{N_30}}$

$VIX = 100\times\sqrt{\left\{ T_1\sigma^2_1  \left[ \frac{N_{T_2}-N_{30}}{N_{T_2}-N_{T_1}} \right] + T_2\sigma^2_2 \left[ \frac{N_{T_{30}}-N_{T_1}}{N_{T_2}-N_{T_1}} \right] \right\}\times\frac{N_{365}}{N_{30}}}$

In [367]:
N_T_1 = M_current + M_settlement_near_term + M_other_near_term
N_T_2 = M_current + M_settlement_next_term + M_other_next_term
N_30 = 30 * 24 * 60
N_365 = 365 * 24 * 60

## Determine VIX

In [370]:
VIX = 100 * math.sqrt(\
                     (T1 * sigma_square_1 * (N_T_2 - N_30) / (N_T_2 - N_T_1) + \
                     T2 * sigma_square_2 * (N_30 - N_T_1) / (N_T_2 - N_T_1)) * \
                     N_365 / N_30)

In [371]:
VIX

1.501496960579997